5.Server and Client based TCP2
=======
---
### 1. Solution of echo_client
### 2. Application Protocol
### 3. I/O Buffer in TCP Socket
### 4. Stop and Wait Method
### 5. TCP Internal Operation Principle

---
#### 1. Solution of echo_client
5.1.1. Problem in echo_server and echo_client
In echo_server, the server have only one purpose that transmit the data received.
TCP is borderless protocol. Even the data is transmitted by dividing by two times of write() or three times of write(), there is no problem.

However, since client write and read the data in sentence by sentence, you have to distinguish the border of data. Terefore this method can be a problem. The funtions read() and write() of TCP doesn't distinguish the border of the data.

5.1.2. Soluton of echo_client
We need to insert the iterative code that read the length of transmitted data through calling the funtion write(). 
```c
#include <unistd.h>

ssize_t write(int fd, const void *buf, size_t count);
/* On success, the number bytes written is returned. On error, -1 is returned. */
ssize_t read(int fd, void *buf, size_t count);
/* On success, the number of bytes read is returned (zero indicates end of file). On error, -1 is returned and errno is set appropriately. */
```
######  Figure 5.1.1. The funtions write and read in UNIX
Following is the code that contains the code of classification required for reading data separately based on TCP.
```c
str_len = write(sock, message, strlen(message));
recv_len = 0;
while (recv_len < str_len) {
	recv_cnt = read(sock, &message[recv_len], BUF_SIZE-1 );
	if ( recv_cnt == -1 ) 
		error_handling("read() error!");
	recv_len += recv_cnt;
}
```
#### 2. Application Protocol
We  will implement a calculator program.
procedure)
1. Server get several numbers and operator information by client.
2. The server calculate based on the information and send the result to client.
	- Ex) information : 3, 5, 9, +
			 result : 3+5+9 = 17
	- EX) information : 4, 3, 2, -
			 result : 4-3-2 = -1

< In UNIX BSD >

< In Windows >


#### 3. I/O Buffer in TCP Socket

#### 4. Stop and Wait Method

#### 5. TCP Internal Operation Principle
